In [1]:
import ee
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection
import plotly
import plotly.express as px
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd

import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4


## Load heatwaves and get initial control dates

In [2]:
daily_max_dic = {}

for state in ['California', 'Oregon', 'Washington', 'British Columbia']:
            
    daily_max_files = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max*')
    
    temp = dd.read_csv(daily_max_files)
    temp = temp.compute()
    daily_max_dic[state] = temp
    

full_df = pd.concat(daily_max_dic.values())

full_df['datetime'] = pd.to_datetime(full_df[['day', 'month', 'year']])
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

full_df['location'] = full_df['county'] + ', ' + full_df['state']
full_df.drop_duplicates(['location', 'datetime'], inplace=True)

full_df = full_df[(full_df['month'] != 10)]
full_df = full_df[(full_df['month'] != 4)]

full_df['pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

In [3]:
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['temp_pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

In [5]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwave_clean_90_v1.csv')
heatwaves = heatwaves[heatwaves['heatwave'] == 1]
#full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/max_temps/max_temps.csv')

heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [5]:
#identify and remove heatwaves that begin in may
may_hw = heatwaves.groupby('heat_ID').apply(lambda x: 5 in x['month'].unique()).reset_index()
may_hw = may_hw[may_hw[0] == True]
heatwaves = heatwaves[~heatwaves['heat_ID'].isin(may_hw['heat_ID'].values)]

In [6]:
#set initial control day to one week prior
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)

In [7]:
#get day of week of each control day
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [8]:
heatwaves = heatwaves[heatwaves['year'] >= 2020]

In [9]:
heatwaves.reset_index(inplace=True)

In [10]:
full_df.reset_index(inplace=True)

In [11]:
# condition that if control day is in heat wave days (the dictionary corresponding to the county)
# and that the temperature of the control day is not 
 

for index, row in heatwaves.iterrows():
    
    i = 0
   
    while (full_df[(full_df['datetime'] == heatwaves.iloc[index, -2]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]>= 0.8):

        if (i == 0) | (i == 1):
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] - timedelta(days=7)
            print(heatwaves.iloc[index, -2])
            print(i)

        if (i == 2):
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] + timedelta(days=28)
            print(heatwaves.iloc[index, -2])

        if (i == 3):
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] + timedelta(days=7)
            print(heatwaves.iloc[index, -2])

        if i >=4:
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] - timedelta(days=7)
            print(heatwaves.iloc[index, -2])
            print(i)
        
        i+=1


San Francisco, California
2020-05-26 00:00:00
2020-05-19 00:00:00
0
Santa Cruz, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
San Joaquin, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Marin, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Alameda, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
San Francisco, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Santa Clara, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Stanislaus, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Sacramento, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Contra Costa, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
San Benito, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Solano, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
Merced, California
2020-05-27 00:00:00
2020-05-20 00:00:00
0
San Benito, California
2020-05-28 00:00:00
2020-05-21 00:00:00
0
Sacramento, California
2020-05-28 00:00:00
2020-05-21 00:00:00
0
Merced, California
2020-05-28 0

In [12]:
heatwaves.head()

,index,Unnamed: 0.1,Unnamed: 0,county,day,month,year,temperature_2m,state,datetime,datetime_ord,location,pct,level_0,heatwave,heat_ID,Heat Wave Duration_x,Heat Wave Duration_y,control_day,day_of_week
0,41,574,5550,San Francisco,2,6,2020,26.468729,California,2020-06-02,737578,"San Francisco, California",0.901961,46.0,1.0,8351b479-476c-4c76-b404-d586a5b62db8,2.0,2.0,2020-05-19,1
1,42,673,6479,Santa Cruz,3,6,2020,26.056117,California,2020-06-03,737579,"Santa Cruz, California",0.952381,48.0,1.0,fc0d4bf7-0320-4ad1-a149-2228a07c9073,2.0,2.0,2020-05-20,2
2,43,593,5709,San Joaquin,3,6,2020,38.130510,California,2020-06-03,737579,"San Joaquin, California",0.956116,45.0,1.0,96af4a5c-7901-497d-b518-94800df75cfe,2.0,2.0,2020-05-20,2
3,44,334,3091,Marin,3,6,2020,26.537195,California,2020-06-03,737579,"Marin, California",0.920635,49.0,1.0,c567c01b-2375-4e76-abb3-0713d0e04db7,2.0,2.0,2020-05-20,2
4,45,50,11,Alameda,3,6,2020,34.180952,California,2020-06-03,737579,"Alameda, California",0.943978,52.0,1.0,9b743974-14fe-490d-8a80-72dda72ee8a6,2.0,2.0,2020-05-20,2


In [13]:
heatwaves = heatwaves.iloc[:, 3:]

In [14]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_clean_90_v1.csv')